<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Logger-Names" data-toc-modified-id="Logger-Names-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Logger Names</a></span></li><li><span><a href="#Logging-on-multiple-sources-at-same-time-(possibly-with-a-single-command)" data-toc-modified-id="Logging-on-multiple-sources-at-same-time-(possibly-with-a-single-command)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Logging on multiple sources at same time (possibly with a single command)</a></span><ul class="toc-item"><li><span><a href="#Avoiding-Multiple-instances" data-toc-modified-id="Avoiding-Multiple-instances-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Avoiding Multiple instances</a></span></li></ul></li><li><span><a href="#Enable/Disable-capture-of-standard-outptut" data-toc-modified-id="Enable/Disable-capture-of-standard-outptut-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Enable/Disable capture of standard outptut</a></span></li><li><span><a href="#Snapshots" data-toc-modified-id="Snapshots-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Snapshots</a></span></li></ul></div>

In [ ]:
# test of the misterious logging module

"D:\dati\ordine\programmazione\pythonprogrammingnotes\logging"

: 

In [ ]:
# %reset

: 

In [25]:
# %load_ext autoreload
# %autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import logging

## Logger Names

Starting from obvious source https://docs.python.org/3/howto/logging.html most minimalistic example:

In [4]:
logging.basicConfig(level=logging.DEBUG)
logging.warning('This is a warning message') 

there are a bunch of operations I can do with these few lines: I have set the level for which we have a log, and i implicitely chose to output on console (I didn't include a filename argument in calling ``.basicConfig``).

I can control output format: 

    logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

Here is a list of format codes which can be used, 
https://docs.python.org/3/library/logging.html#logrecord-attributes,
you can even modify colors or font formats (on supported terminals).

For example, next lines modify format of WARNING messages only (bold, red), works e.g. in IPython terminal. 

In [26]:
logging.basicConfig(format='%(asctime)s %(message)s') #this applies to current level DEBUG, but doens't work
logging.addLevelName( logging.WARNING, "\033[1;31m%s\033[1;0m" % logging.getLevelName(logging.WARNING))

logging.warning("This is a warning message")
logging.error("This is an error message")
logging.debug("This is an debug message")

2022-05-23 15:43:50,351 This is a warning message
2022-05-23 15:43:50,354 This is an error message


Howevever this not always works. For example, adding time in the format didn't work (they would work if run as first lines). In other words, trying to redefine format or level by calling basicconfig on an already configured logging module is not a good idea. 

In [28]:
# this works only if it is the first time you run basicConfig.

import logging
logging.basicConfig(format='%(asctime)s %(message)s')
logging.warning('is when this event was logged.')

2022-05-23 15:44:00,516 is when this event was logged.


In [10]:
# DAvvero non funziona su nessun livello

import logging
logging.basicConfig(format='%(asctime)s %(message)s',level=logging.DEBUG)

logging.debug('DEBUG')
logging.info('Info')
logging.warning('is when this event was logged.')
logging.error('is when this event was logged.')

DEBUG:root:DEBUG
INFO:root:Info
ERROR:root:is when this event was logged.


In [31]:
# Mentre questo

import logging
logging.addLevelName( logging.DEBUG, '%(asctime)s %(message)s') 
#logging.addLevelName( logging.WARNING, "\033[1;31m%s\033[1;0m" % logging.getLevelName(logging.WARNING)) #funziona colore

logging.debug('DEBUG')
logging.info('Info')
logging.warning('is when this event was logged.')
logging.error('is when this event was logged.')

%(asctime)s %(message)s:root:DEBUG
INFO:root:Info
ERROR:root:is when this event was logged.


Note also that I am calling directly ``logging``, meaning I am working on the same instance of logger. As stated in the tutorial, this makes it difficult to understand from where the logger is called.

In [5]:
import logging

def logging_function():
    print("\nIn Logging Function:")
    print("logging: ",logging)
    l = logging.getLogger()
    print("name",__name__)
    print('logger:',l)
    
print(__name__)
print("logging: ",logging)

print("logger: ",logging.getLogger())

logging_function()


__main__
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger:  <RootLogger root (WARNING)>

In Logging Function:
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
name __main__
logger: <RootLogger root (WARNING)>


Note that calling a logger inside a function from a cell, gives a single root __name__ = "__main__" logger and the functions are not recognized, e.g.:

In [20]:
# questo e' l'errore mortale
import logging

def logging_function():
    print("\nIn Logging Function:")
    print("logging: ",logging)
    l = logging.getLogger(__name__)
    print('logger:',l,logging.getLogger())
    
print(__name__)
print("logging: ",logging)

print("logger: ",logging.getLogger(__name__),logging.getLogger())

logging_function()


__main__
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger:  <Logger __main__ (WARNING)> <RootLogger root (WARNING)>

In Logging Function:
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger: <Logger __main__ (WARNING)> <RootLogger root (WARNING)>


In [14]:


from logger import logging_function as lf
    
print(__name__)
print("logging: ",logging)

print("logger: ",logging.getLogger(__name__),logging.getLogger())

lf()


__main__
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger:  <Logger __main__ (WARNING)> <RootLogger root (WARNING)>

In Logging Function:
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger: <Logger logger (WARNING)>


This import same code as previous cell from an external file, note that function names are correctly logged.

## Logging on multiple sources at same time (possibly with a single command)

For a finer tuning, StramHandlers can be manipulated.

However this textbook example, if launched multiple times add an handler every time:

In [ ]:
import logging

# create logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# add formatter to ch
ch.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

# add ch to logger
logger.addHandler(ch)

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

: 

In [ ]:
print(logger)
print(ch)

: 

In [52]:
logger.handlers

[<StreamHandler stderr (DEBUG)>,
 <StreamHandler stderr (DEBUG)>,
 <StreamHandler stderr (DEBUG)>]

### Avoiding Multiple instances

In [65]:
def reset_logger(logger):
    """remove all handlersr in place. """
    for i in range(len(logger.handlers)):
        logger.handlers.pop()
        
reset_logger(logger)

In [75]:
from dataIO.logger import reset_logger
reset_logger(logger)


ImportError: cannot import name 'reset_logger' from 'dataIO.logger' (c:\users\kovor\documents\python\package_demo\source\dataIO\logger.py)

In [66]:
logger.handlers

[]

In [71]:
# Questo era in dataIO.logger e trattava un problema che mi 
# avveniva spesso in passato su notebook in cui avevo definito handlers 
# multipli per loggare sia su file che su console

import logging


def start_logger(cfgfile=None):

    #if logger exists (this is based only in checking it has some handler) 
    # recover the current logger without adding handlers, if not load from file
    #  or if config file is not provided, uses default options for logger
       
    print("N: ",__name__) # qui mostra non serve perche sara start_logger    
    try:
        logger
        print("logger %s existing"%logger)
    except:
        print("logger non existing.")
        
    logger=logging.getLogger()
    if len(logger.handlers) == 0:
        """not sure about original indentation here."""   
        if cfgfile is not None:
            logger=logging.config.fileConfig(cfgfile)
        else:
            logger.  #set default     
            
            #print('cfgfile missing, and no logger existing, not sure what happens now.')
            
    
    return logger

In [72]:
logger = start_logger()
# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

warn message
error message
critical message


N:  __main__
logger non existing.
cfgfile missing, not sure what happens now.


In [62]:
from dataIO.logger import start_logger

logger = start_logger()
# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

2022-05-23 16:13:44,760 warn message
2022-05-23 16:13:44,762 error message
2022-05-23 16:13:44,764 critical message


logger non existing.


In [58]:
logger.handlers

[<StreamHandler stderr (NOTSET)>]

## Enable/Disable capture of standard outptut

## Snapshots